In [1]:
def get_performance(strat):
    returns_analysis = strat.analyzers.getbyname('returns').get_analysis()
    anualreturns_analysis = strat.analyzers.getbyname('annualreturn').get_analysis()
    drawdown_analysis = strat.analyzers.getbyname('drawdown').get_analysis()
    sharpe_analysis = strat.analyzers.getbyname('sharperatio').get_analysis()
    trade_analysis = strat.analyzers.getbyname('tradeanalyzer').get_analysis()
    timedrawdown_analysis = strat.analyzers.getbyname('timedrawdown').get_analysis()
    
    columns = [
        'total return',
        'average return',
        'normalized return',
        'max drawdown',
        'money_drawdown',
        'time_drawdown',
        'sharpe_ratio',
        'total_trades',
        'total_won',
        'won _avg',
        'total_lost',
        'avg_lost',
        'net_pnl_trade',
    ]

    values = [
        returns_analysis['rtot'],
        returns_analysis['ravg'],
        returns_analysis['rnorm100'],
        drawdown_analysis['max']['drawdown'],
        drawdown_analysis['max']['moneydown'],
        timedrawdown_analysis['maxdrawdownperiod'],
        sharpe_analysis['sharperatio'],
        trade_analysis['total']['total'],
        trade_analysis['won']['total'],
        trade_analysis['won']['pnl']['average'],
        trade_analysis['lost']['total'],
        trade_analysis['lost']['pnl']['average'],
        trade_analysis['pnl']['net']['average'],
    ]


    performance = pd.DataFrame({'metric':columns, 'value':values})
    performance = performance.set_index('metric')


    analysis = strat.analyzers.getbyname('transactions').get_analysis()
    rows = []
    # Iterar sobre el OrderedDict y extraer los datos
    for date, transactions in analysis.items():
        for transaction in transactions:
            # Desempaquetar los valores de cada transacción
            amount, price, sid, symbol, value = transaction
            # Crear una fila con la fecha y los valores
            rows.append([date, amount, price, sid, symbol, value])

    # Crear un DataFrame con las filas y las columnas especificadas
    trades = pd.DataFrame(rows, columns=['date', 'amount', 'price', 'sid', 'symbol', 'value'])

    # Mostrar el DataFrame

    return performance, trades

In [3]:
dji = yf.Ticker("^DJI").history(period="max")
vix = yf.Ticker("^VIX").history(period="max")
sp500 = yf.Ticker("^SPX").history(period="max")

In [28]:
import backtrader as bt

class VixRSI(bt.Strategy):
    params = (
        ('sma_period', 200),  # Periodo de la media móvil rápida
        ('vix_period', 10),  # Periodo de la media móvil rápida
        ('rsi_period', 2),  # Periodo de la media móvil lenta
        ('threshold', 0.05),  # Umbral del 5%

    )

    def __init__(self):
        self.sma = bt.talib.SMA(self.datas[0].close, timeperiod=self.params.sma_period)
        self.rsi = bt.talib.RSI(self.datas[0].close, timeperiod=self.params.rsi_period)

        self.vix_rsi = bt.talib.RSI(self.datas[1].close, timeperiod=self.params.rsi_period)
        self.vix_sma = bt.talib.SMA(self.datas[1].close, timeperiod=self.params.vix_period)

        self.last_buy_date = None

        self.instrument_data = self.datas[0]
        self.vix_data = self.datas[1]

        self.order = None

    def next(self):
        actual_date = self.instrument_data.datetime.datetime(0)

        # Si hay una posición abierta
        if self.position:
            time_in_position = (actual_date - self.last_buy_date).days

            # Cierra la posición si lleva más de 5 días o si el RSI supera 65
            if time_in_position >= 30: # or self.rsi > 90:
                self.close(data=self.instrument_data)

        else:
            # Obtén los precios y valores de los indicadores
            sp500_close = self.instrument_data.close[0]
            vix_ma_value = self.vix_sma[0]
            vix_close = self.vix_data.close[0]

            # Verifica si el VIX está 5% por encima de su media móvil
            vix_above_ma = vix_close > (vix_ma_value * (1 + self.params.threshold))

            # Verifica si el precio del S&P500 es un nuevo mínimo de los últimos 10 periodos
            is_lower_low = sp500_close < min(self.instrument_data.close.get(size=10)[:-1])

            # Suma del RSI actual y el anterior del VIX
            cum_rsi_vix = self.vix_rsi[0] + self.vix_rsi[-1]

            # Verifica si hay una orden pendiente
            # Condiciones de compra
            if not self.order and vix_above_ma and sp500_close > self.sma and cum_rsi_vix >= 150 and is_lower_low:
                self.order = self.buy(
                    price=sp500_close * 0.97,  # Precio de la orden Limit
                    data=self.instrument_data,
                    size=1,
                    exectype=bt.Order.Limit  # Orden de tipo Limit
                )

                # Guarda la fecha de la compra
                self.last_buy_date = actual_date


cerebro = bt.Cerebro()

# for ticker in symbols:
#     data_feed = bt.feeds.PandasData(dataname=symbols[ticker][16408])
#     cerebro.adddata(data_feed, name=ticker)

data_feed = bt.feeds.PandasData(dataname=sp500)
data_feed2 = bt.feeds.PandasData(dataname=vix)
cerebro.adddata(data_feed, name='spx')
cerebro.adddata(data_feed2, name='vix')

cerebro.addstrategy(VixRSI)

cerebro.broker.setcash(5000)
cerebro.broker.setcommission(
    leverage=20,
    commission=7e-4,
)

cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.AnnualReturn)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.TimeDrawDown)
cerebro.addanalyzer(bt.analyzers.SharpeRatio)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.Transactions)

print(f'start cash: {cerebro.broker.cash}')

results = cerebro.run()
strat = results[0]

cerebro.plot(figsize=(18, 10), iplot=False)

print(f'final cash: {cerebro.broker.cash}')
performance, transactions = get_performance(strat)

start cash: 5000
final cash: 58.36906218674185


In [ ]:
performance

In [ ]:
transactions

In [7]:
sp500_4h = yf.Ticker("^SPX").history(period='max', interval='1h')